In [ ]:
# @title  Whisper Turbo Fine-Tuning with LoRA (Whisper large-v3-turbo)

# 1. Install dependencies
# !pip install -q transformers librosa accelerate peft bitsandbytes

# !pip install jiwer

# !pip install --upgrade datasets==3.6.0


In [ ]:
# from huggingface_hub import login
# login()

In [1]:
import pandas as pd
import librosa
import glob
import os

In [ ]:
files1 = glob.glob(r"F:\Hybridmodel-project\dataset\asr\th/*tsv")
files2 = glob.glob(r"F:\Hybridmodel-project\dataset\asr\th/*tsv")
# column_names = ["id","path","transcription","raw_transcription","phoneme","lenght","gender"]
for file in files1:
    # data = pd.read_csv(file,sep="\t",names=column_names, header=None)
    data = pd.read_csv(file,sep="\t")
    data = data.rename(columns = {"sentence":"transcription"})
    data[["client_id","path","sentence_id","transcription"]].reset_index(drop=True).to_csv(file.replace(".tsv",".csv"),index=False)

In [105]:
files = [r"F:\Hybridmodel-project\dataset\asr\fluers_th\train.csv",
         r"F:\Hybridmodel-project\dataset\asr\th\train.csv"]
# validated
train = pd.DataFrame()
for i in files:
    train = pd.concat([train,pd.read_csv(i)])

In [106]:
def transform(x):
    if "common" in x:
        x = "F:/Hybridmodel-project/dataset/asr/th/train/th_train_0/"+x
        
    else:
        x = "F:/Hybridmodel-project/dataset/asr/fluers_th/audio/train/"+x
    return x
train["path"] =train["path"].apply(lambda x:transform(x))
train["exits"] = train["path"].apply(lambda x:os.path.exists(x))

In [107]:
train[train["exits"] == True][["path","transcription"]].to_csv("train_all.csv",index=False)

In [2]:
# 2. Enable GPU
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# 3. Load dataset (example: Common Voice English; replace with your dataset)
from datasets import load_dataset
# dataset = load_dataset("mozilla-foundation/common_voice_17_0", "th",trust_remote_code=True, split = {
#     "train": "train[:30%]",
#     "validation": "validation[:30%]"
# })
# save_path  = r"F:\Hybridmodel-project\dataset\asr\cv17"
dataf = {"train":"train_all.csv","eval":"eval_all.csv"}
dataset = load_dataset("csv", data_files=dataf,split={"train":"train[:1%]","eval":"eval[:1%]"})
# eval = load_dataset("csv", data_files=r"eval_all.csv")




# dataset.save_to_disk(save_path)
model_path = "../ipyfiles/model/whisper-large-v3-turbo"
# Preprocessing
import librosa
from transformers import WhisperFeatureExtractor, WhisperTokenizer
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path)




Using device: cuda


In [3]:
# train2 = train.rename_columns({"path":"input_features","transcription":"labels"})

train = dataset["train"].rename_columns({"path":"input_features","transcription":"labels"})
eval = dataset["eval"].rename_columns({"path":"input_features","transcription":"labels"})

In [ ]:
def prep(batch):
    audio = batch["path"]

    audio_array,fs = librosa.load(audio,sr=16000)


    batch["input_features"] = feature_extractor(audio_array, sampling_rate = 16000, return_tensors="pt").input_features[0]
    with tokenizer.as_target_tokenizer():
        batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

train = dataset[0].map(prep, remove_columns=dataset[0].column_names)
eval = dataset[1].map(prep, remove_columns=dataset[1].column_names)


In [4]:
import evaluate
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

In [5]:
# 4. Load model with PEFT-LoRA
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05,target_modules = ["q_proj","v_proj"])
model = get_peft_model(model, peft_config)



In [6]:
class Datacollator:
    def __init__(self,feature_extractor,label_pad_token_id,tokenizer):
        self.feature_extractor = feature_extractor
        self.label_pad_token_id = label_pad_token_id
        self.tokenizer = tokenizer
        self.padding= "longest"
    def __call__(self,data):
        
        input_features = [{"input_features": librosa.load(f["input_features"],sr=16000)[0]} for f in data]
        label_features = [{"input_ids": tokenizer(f["labels"],return_tensors="pt").input_ids} for f in data]
        
        # audio_array,fs = librosa.load(path,sr=16000)
        
        input_features = [{"input_features": feature_extractor(f["input_features"], sampling_rate = 16000, return_tensors="pt").input_features[0]} for f in input_features]
        
        
        batch_inputs = self.feature_extractor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt"
        )
        
        batch_labels = self.tokenizer.pad(
            label_features,
            padding=self.padding,
            return_tensors="pt"
        )
        
        
        labels = batch_labels["input_ids"].masked_fill(
            batch_labels["input_ids"] == self.tokenizer.pad_token_id, -100
        )

        batch = {
            "input_features": batch_inputs["input_features"],
            "labels": labels
        }
        # print(batch)
        
        return  batch
        

In [ ]:
# 5. Training setup
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import os
os.environ["WANDB_DISABLED"] = "True"
training_args = Seq2SeqTrainingArguments(
    output_dir="whisper-turbo-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    num_train_epochs=1,
    optim="adamw_torch",
    logging_steps=10,
    save_steps=200,
    fp16=True,
    push_to_hub=False,
    eval_strategy="steps",
    eval_steps=200,
    label_names=["labels"]
)

data_collator = Datacollator(feature_extractor, label_pad_token_id=tokenizer.pad_token_id,tokenizer=tokenizer)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    tokenizer=feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# 9. Train
trainer.train()

# 10. Evaluate
metrics = trainer.evaluate()
print("Evaluation Results:", metrics)

# 11. Save final model
model.save_pretrained("whisper_turbo_finetuned")
# processor.save_pretrained("whisper_turbo_finetuned")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
C:\Users\wayupuk sommuang\AppData\Local\Temp\ipykernel_8324\3678688535.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


In [22]:
train2 = train.rename_columns({"path":"input_features","transcription":"labels"})

In [37]:
data_collator = Datacollator(feature_extractor, label_pad_token_id=tokenizer.pad_token_id)

In [61]:
train2

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 354
})

In [78]:
tokenizer(train2[0]["labels"],return_tensors="pt").input_ids

tensor([[50258, 50364,  4131,  4294,  4943, 48556,  6033,  7044,  9345, 23398,
         11099, 12774,  5914,    94, 30121,  5104,  8163, 15417, 38206, 43988,
         11851, 29560,  4294,  4131,  9345,  6033,  7643,  6033,  9345,  6033,
           220, 20512,  5104, 10196, 20512, 18970,  6192,  5981,  4131,  6192,
          5981,  6223, 28417, 28955, 13687,  6223, 11526,  5981,  9345, 30739,
          9345,  4750, 39507,   220, 22214,  8055, 28955,  4750,  4943,  7643,
          9596,  9345,  6033, 24929, 26346, 27216,  4131,  6192,  6033,  8163,
         24865, 10196,  7067,  7067,  7044, 23398, 11099, 12774, 19245, 37805,
         15412,  6223, 10133,  4750, 20512,  5104, 10196, 10133,  4294, 26134,
         24815,  6223,  5914,   223,  9596,  4131, 30739, 44579, 12055, 28417,
         19417,  4294, 13715,  4943, 24815,  7067, 28955, 35782, 12055,  4131,
         25977,  4750, 40050,  9345, 50257]])